In [1]:
from glob import glob 
from fastcore.helpers import load_pickle
import pandas as pd 
import numpy as np
import random 

In [2]:
all_res = glob('results/inverse_photoswitch/*pkl')

In [3]:
all_res

['results/inverse_photoswitch/2022-09-02-20-00-50_metrics.pkl',
 'results/inverse_photoswitch/2022-09-02-19-35-07_metrics.pkl',
 'results/inverse_photoswitch/2022-09-02-19-54-20_metrics.pkl',
 'results/inverse_photoswitch/2022-09-02-19-28-36_metrics.pkl',
 'results/inverse_photoswitch/2022-09-02-20-13-53_metrics.pkl',
 'results/inverse_photoswitch/2022-09-02-19-48-00_metrics.pkl',
 'results/inverse_photoswitch/2022-09-02-20-20-26_metrics.pkl',
 'results/inverse_photoswitch/2022-09-02-19-41-24_metrics.pkl',
 'results/inverse_photoswitch/2022-09-02-20-27-07_metrics.pkl',
 'results/inverse_photoswitch/2022-09-02-20-07-19_metrics.pkl']

In [4]:
d = load_pickle(all_res[0])

In [13]:
d

{'meta': {'temperature': 1.5,
  'max_tokens': 80,
  'model': 'ada:ft-lsmoepfl-2022-09-02-18-11-44'},
 'predictions': array(['CC1=NOC(C)=C1/N=N/C2=C(C)C(C)C2',
        'CCN(CCC#N)C(C=C%21C)=CC=C%21/N=N/C%22=CC=CC=C%22',
        'CCN(CCC#N)C%16=CC=C(OC)35=C%16/N=N/C%17=CC=C(NACC#N)C=C%17OC%32',
        'CC%11=CC(#N)=CC=C%11/N=N/C%12=CC([N+]([O-])=O)C=C%12',
        'FC1=CC=CC(/N=N/C2=CC=C(TA=C2)F3G3D4', 'Am[N]C9=CC([S?',
        '17Occc1f(O)c(/N=N/C2=CC(O)=NT2)=CC(F)=C1',
        'OCNVKnight:/C=CSR(/N=N/C', 'CCc1nncntt01*nnnn*,C(C=C1C)N=NC2NC',
        'CC(C=CC=C1C)=C1/N=N/C2=C(NC(C)=O)C=CC=C2',
        'Car2=CC1=CC=C(/N=N/C2=CC=CC=C2)C=C1', 'C', 'ClC1=NOC(C)=N',
        'CCN(CC)C(C=C1)=CC=C1/N=N/C2=CC(CCC#ani', 'CC',
        'CN-C(C=C%19)=CC=C%19/N=', 'ClC(cnC1ncs1',
        'OISSRSC(C=C8)=CC=C8/N=N/C9=CC=CC=C9',
        'N#CCCN()C[O-iliated]C%13=C(N=C(N=NC%14=CS%15=CC=C([N+]([C',
        'ASC=', 'oc%21=N=CC_(N=C(',
        'Samsung0197HNF0181CC(NC)N=N-C02=CC(N=CC=C2)=C([N+]([O-])=O)-C=

In [14]:
results = []

for file in all_res: 
    res = load_pickle(file)

    for d in res: 
        compiled_res = {
            'temperature': d['meta']['temperature'],
            'frac_valid_smiles': d['fractions_valid_smiles'],
            'fractions_smiles_in_train': d['fractions_smiles_in_train'],
            'n_pi_star_mae': d['n_pi_star_metrics']['mean_absolute_error'], 
            'pi_pi_star_mae': d['pi_pi_star_metrics']['mean_absolute_error'],
            'mol_similarity_metrics_mean': d['mol_similarity_metrics_mean']['mean'],
        }
        results.append(compiled_res)

In [15]:
results = pd.DataFrame(results)

In [16]:
aggregated_results = results.groupby('temperature').agg(['mean', 'std'])

In [17]:
"\SI{{{frac_valid_smiles:.2f}}}\pm{{frac_valid_smiles_std:.2f}}".format(frac_valid_smiles=1, frac_valid_smiles_std=2)

'\\SI{1.00}\\pm{frac_valid_smiles_std:.2f}'

In [18]:
ROW_TEMPLATE = "{temperature} & \SI{{{frac_valid_smiles:.2f}}}\pm{{{frac_valid_smiles_std:.2f}}} & \SI{{{frac_smiles_on_train:.2f}}}\pm{{{frac_smiles_in_train_std:.2f}}} & \SI{{{mol_similarity:.2f}}}\pm{{{mol_similarity_std:.2f}}} & \SI{{{pi_pi_star_mae:.2f}}}\pm{{{pi_pi_star_mae_std:.2f}}} & \SI{{{n_pi_star_mae:.2f}}}\pm{{{n_pi_star_mae_std:.2f}}} \\\\"

In [22]:
rows = [    ]
for i, row in aggregated_results.iterrows():
    row = row.round(2)
    rows.append(ROW_TEMPLATE.format(
        temperature=i,
        frac_valid_smiles=row['frac_valid_smiles']['mean'],
        frac_valid_smiles_std=row['frac_valid_smiles']['std'],
        frac_smiles_on_train=row['fractions_smiles_in_train']['mean'],
        frac_smiles_in_train_std=row['fractions_smiles_in_train']['std'],
        mol_similarity=row['mol_similarity_metrics_mean']['mean'],
        mol_similarity_std=row['mol_similarity_metrics_mean']['std'],
        pi_pi_star_mae=row['pi_pi_star_mae']['mean'],
        pi_pi_star_mae_std=row['pi_pi_star_mae']['std'],
        n_pi_star_mae=row['n_pi_star_mae']['mean'],
        n_pi_star_mae_std=row['n_pi_star_mae']['std'],
    ))

In [23]:
print("\n".join(rows).replace('}\pm{', '\pm ').replace('SI', 'num'))

0.0 & \num{1.00\pm 0.00} & \num{0.81\pm 0.32} & \num{0.33\pm 0.01} & \num{32.61\pm 4.71} & \num{21.76\pm 4.39} \\
0.25 & \num{1.00\pm 0.01} & \num{0.80\pm 0.19} & \num{0.32\pm 0.01} & \num{36.20\pm 4.67} & \num{20.44\pm 3.68} \\
0.5 & \num{0.99\pm 0.02} & \num{0.65\pm 0.11} & \num{0.31\pm 0.01} & \num{41.76\pm 5.10} & \num{19.98\pm 3.41} \\
0.75 & \num{0.97\pm 0.03} & \num{0.55\pm 0.06} & \num{0.30\pm 0.01} & \num{44.67\pm 6.52} & \num{20.05\pm 3.80} \\
1.0 & \num{0.79\pm 0.07} & \num{0.44\pm 0.08} & \num{0.28\pm 0.01} & \num{46.38\pm 9.58} & \num{20.21\pm 4.47} \\
1.25 & \num{0.49\pm 0.07} & \num{0.32\pm 0.11} & \num{0.28\pm 0.01} & \num{45.69\pm 13.76} & \num{18.66\pm 4.14} \\
1.5 & \num{0.18\pm 0.05} & \num{0.20\pm 0.14} & \num{0.22\pm 0.04} & \num{45.43\pm 17.47} & \num{18.29\pm 10.16} \\


## Now sample a couple of molecules for validation with TDDFT

Let's do it now with one file

In [49]:
all_res[0]

'results/inverse_photoswitch/2022-09-02-20-00-50_metrics.pkl'

In [ ]:
Which comes from 

In [51]:
d  = load_pickle(all_res[0])

In [53]:
d[0]['meta']['model']

'ada:ft-lsmoepfl-2022-09-02-18-05-09'

In [63]:
smiles_to_sample_from = {}

for subd in d:
    smiles= subd['predictions'][subd['valid_smiles']][~np.array(subd['smiles_in_train'])]
    smiles_to_sample_from[subd['meta']['temperature']] = smiles

In [67]:
sampled_smiles = {}

for t, smiles in smiles_to_sample_from.items():
    try:
        sampled_smiles[t] = random.sample(list(smiles), 3)
    except Exception:
        pass

In [69]:
sampled_smiles_all = []
for _, s in sampled_smiles.items():
    sampled_smiles_all.extend(s)

for s in sampled_smiles_all:
    print(s)

CN(C)C1=CC=C(/N=N/C2=CC=C(N(C)C)C=C2)C=C1
O=[N+]([O-])C1=CC=C(/N=N/C2=CC=C(N(C)C)C=C2)C=C1
CC1=C(C(C)=NN1)/N=N/C2=CC=CC=C2
O=C(O)C1=CC=C(/N=N/C2=CC=CC=C2)C=C1
OC%28=C(N=CC=C%29)C%29=C(/N=N/C%30=CC=C(C(C)=O)C=C%30)C=C%28
CCN(CC)C(C=C1)=CC=C1/N=N/C2=CC=C(N(CC)CC)C=C2
NC1=CC=CC=C1/N=N/C2=CC=CC=C2
CC1=C(/N=N/C2=C(C)C=CC=C2C#N)C(C)=NO1
OC%23=C(N=CC=C%24)C%24=C(/N=N/C%25=CC=CC=C%25)C=C%23
OC1=C(/N=N/C2=CC=CC=C2)C=CC=C1
CC1=NOC(C)=C1/N=N/C2=CC=C(C(O)=O)C=C2
BrC1=CC=CC(/N=N/C2=CC=C(NCCC#N)C=C2)=CC=C1
CN(C=C1C)C=C1/N=N/C2=C(Br)C=CC=C2Br
[H]C7=CC(N(CCO)CCC#N)=CC=C7/N=N/C8=CC=C([N+]([O-])=O)C=C8
[H]N(C)C1=CC=C(/N=N/C2=CC=CN=C2)C=C1
CN(OB)C1=CC(NC(C)=O)=CC=C1/N=N/C2=CC=C([N+]([O-])=O)C=C2
CC(C=C(N(CCC#N)CCO)C=C1)=C1/N=N/C2=C(OC)C=C(OC)C=C2
FC%21=CC(S)=CC(F)=C%21/N=N/C%22=CC=CC=C%22
